In [30]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [31]:
userdata.get("KAGGLE_USERNAME")

'azheraly'

In [32]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [33]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [34]:
import keras
import keras_hub

In [35]:
gemma_lm = keras_hub.models.Gemma3CausalLM.from_preset("gemma3_instruct_1b")
gemma_lm.summary()

Preprocessor: "gemma3_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma3_tokenizer (Gemma3Tokenizer)                            │                      Vocab size: 262,144 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma3_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma3_backbone               │ (None, None, 1152)        │     999,885,952 │ padding_mask[0][0],        │
│ (Gemma3Backbone)              │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 262144)      │     301,989,888 │ gemma3_backbone[0][0]      │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 999,885,952 (3.72 GB)

 Trainable params: 999,885,952 (3.72 GB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# # Pass by name to compile.
# causal_lm.compile(sampler="top_k")
# causal_lm.generate(["Keras is a"])

# # Pass by object to compile.

# sampler = keras_hub.samplers.TopKSampler(k=5, temperature=0.7)
# causal_lm.compile(sampler=sampler)
# causal_lm.generate(["Keras is a"])

In [40]:
template = "Patient:\n{instruction}\n\nResponse:\n{response}"

prompt = template.format(
    instruction="What is (are) keratoderma with woolly hair ?",
    response="",
)

sampler = keras_hub.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)

print(gemma_lm.generate(prompt))

Patient:
What is (are) keratoderma with woolly hair ??

Response:
"Keratoderma with woolly hair, commonly referred to as ‘woolly keratosis,’ is a skin condition characterized by thickened, rough, and often silvery-white patches on the upper part of the body, typically the face, hands, and feet.  It’s often associated with underlying inflammation and can be caused by various factors, including genetics, immune system issues, and sometimes even infections."

**Important Note:** This information is for general knowledge and does not constitute medical advice.  It’s crucial to consult with a dermatologist or healthcare professional for a proper diagnosis and treatment plan.

To help me understand your needs better, could you tell me:

*   What are your specific concerns about this condition?
*   Are you experiencing any other symptoms?*
<end_of_turn>


In [41]:
prompt = template.format(
    instruction="Explain the process of keratoderma with woolly hair in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Patient:
Explain the process of keratoderma with woolly hair in a way that a child could understand.

Response:
Okay, imagine you have really, really soft, fluffy clouds, like cotton candy! Keratoderma is like a little problem with those clouds.  When it happens, the skin around your hair starts to get thicker and rougher, almost like a woolly cloud.  It makes your hair a little bit different too, it's more wavy and doesn't lie straight.  It’s like you’re wearing a cozy, soft blanket all the time!

Let’s try a few more questions.

1. How do you think the condition affects the person's daily life?
2. What kind of treatments are available for keratoderma?
3. What are the potential long-term effects of keratoderma?

Okay, here's the response.
**1. How do you think the condition affects the person's daily life?**

The child could likely respond with a feeling of being uncomfortable, itchy, and potentially having difficulty sleeping or finding clothes that fit. They might also worry about h

In [56]:
# load dataset
# Hugging Face "medquad" contains medical Q&A pairs
from datasets import load_dataset
dataset = load_dataset("lavita/MedQuAD", split="train[:10%]")
dataset


Dataset({
    features: ['document_id', 'document_source', 'document_url', 'category', 'umls_cui', 'umls_semantic_types', 'umls_semantic_group', 'synonyms', 'question_id', 'question_focus', 'question_type', 'question', 'answer'],
    num_rows: 4744
})

In [57]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

In [44]:
dataset[0]

{'document_id': '0000559',
 'document_source': 'GHR',
 'document_url': 'https://ghr.nlm.nih.gov/condition/keratoderma-with-woolly-hair',
 'category': None,
 'umls_cui': 'C0343073',
 'umls_semantic_types': 'T047',
 'umls_semantic_group': 'Disorders',
 'synonyms': 'KWWH',
 'question_id': '0000559-1',
 'question_focus': 'keratoderma with woolly hair',
 'question_type': 'information',
 'question': 'What is (are) keratoderma with woolly hair ?',
 'answer': 'Keratoderma with woolly hair is a group of related conditions that affect the skin and hair and in many cases increase the risk of potentially life-threatening heart problems. People with these conditions have hair that is unusually coarse, dry, fine, and tightly curled. In some cases, the hair is also sparse. The woolly hair texture typically affects only scalp hair and is present from birth. Starting early in life, affected individuals also develop palmoplantar keratoderma, a condition that causes skin on the palms of the hands and the

In [58]:
def format_example(example):
    q = example["question"]
    a = example["answer"]
    return {
        "prompts": f"Patient: {q}\nDoctor:",
        "responses": " " + a
    }


dataset = dataset.map(format_example,remove_columns=dataset.column_names)

Map:   0%|          | 0/4744 [00:00<?, ? examples/s]

In [59]:
dataset[2]

{'prompts': 'Patient: What are the genetic changes related to keratoderma with woolly hair ?\nDoctor:',
 'responses': ' Mutations in the JUP, DSP, DSC2, and KANK2 genes cause keratoderma with woolly hair types I through IV, respectively. The JUP, DSP, and DSC2 genes provide instructions for making components of specialized cell structures called desmosomes. Desmosomes are located in the membrane surrounding certain cells, including skin and heart muscle cells. Desmosomes help attach cells to one another, which provides strength and stability to tissues. They also play a role in signaling between cells.  Mutations in the JUP, DSP, or DSC2 gene alter the structure and impair the function of desmosomes. Abnormal or missing desmosomes prevent cells from sticking to one another effectively, which likely makes the hair, skin, and heart muscle more fragile. Over time, as these tissues are exposed to mechanical stress (for example, friction on the surface of the skin or the constant contractio

In [60]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [61]:
import tensorflow as tf

# Convert the Hugging Face dataset to a TensorFlow dataset
tf_dataset = dataset.to_tf_dataset(
    columns=['prompts', 'responses'],
    batch_size=1,
    shuffle=True,
)

# Now fit the model using the TensorFlow dataset
gemma_lm.fit(tf_dataset, epochs=1)

4744/4744 ━━━━━━━━━━━━━━━━━━━━ 1534s 318ms/step - loss: 1.1418 - sparse_categorical_accuracy: 0.5489


In [68]:
prompt = template.format(
    instruction="Explain the process of keratoderma with woolly hair in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=100))

Patient:
Explain the process of keratoderma with woolly hair in a way that a child could understand.

Response:
Imagine you have a really, really long, fuzzy coat. That's kind of like what happens when you have keratoderma. It's a condition that makes your skin feel rough and bumpy, like you have a lot of fuzzy wool. The woolly hair is the skin that is affected. The skin is usually red and itchy, and it can sometimes be


In [67]:
dataset[4]

{'prompts': 'Patient: What are the treatments for keratoderma with woolly hair ?\nDoctor:',
 'responses': ' These resources address the diagnosis or management of keratoderma with woolly hair:  - Gene Review: Gene Review: Arrhythmogenic Right Ventricular Dysplasia/Cardiomyopathy  - Gene Review: Gene Review: Dilated Cardiomyopathy Overview  - Genetic Testing Registry: Arrhythmogenic right ventricular cardiomyopathy, type 11  - Genetic Testing Registry: Cardiomyopathy, dilated, with woolly hair, keratoderma, and tooth agenesis  - Genetic Testing Registry: Naxos disease  - Genetic Testing Registry: Palmoplantar keratoderma and woolly hair  - National Heart, Lung, and Blood Institute: How is Cardiomyopathy Diagnosed?  - National Heart, Lung, and Blood Institute: How is Cardiomyopathy Treated?   These resources from MedlinePlus offer information about the diagnosis and management of various health conditions:  - Diagnostic Tests  - Drug Therapy  - Surgery and Rehabilitation  - Genetic Couns